# Consumer Price Index (CPI) Prediction
Building an RNN model to predict the CPI for month of September 2024. This Model leverages all the CPI data available since 1913. The Data was obtained from Bureau Of Labor Statistics (USA) on the month of September 2024, thus it has the CPI data till August 2024.

I am using Recurrent Neural Network (RNN), a deep learning model to predict the September CPI.


Import Packages

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.models import load_model


#### Import Data

In [4]:
dataset = pd.read_csv('data/CPISeries.csv')

In [5]:
print(dataset.head())

   Year   Jan   Feb   Mar   Apr   May   Jun   Jul   Aug   Sep   Oct   Nov  \
0  1913   9.8   9.8   9.8   9.8   9.7   9.8   9.9   9.9  10.0  10.0  10.1   
1  1914  10.0   9.9   9.9   9.8   9.9   9.9  10.0  10.2  10.2  10.1  10.2   
2  1915  10.1  10.0   9.9  10.0  10.1  10.1  10.1  10.1  10.1  10.2  10.3   
3  1916  10.4  10.4  10.5  10.6  10.7  10.8  10.8  10.9  11.1  11.3  11.5   
4  1917  11.7  12.0  12.0  12.6  12.8  13.0  12.8  13.0  13.3  13.5  13.5   

    Dec  
0  10.0  
1  10.1  
2  10.3  
3  11.6  
4  13.7  


In [6]:
# Converting the data to long format
dataset_melt = pd.melt(dataset, id_vars=['Year'], var_name='Month', value_name='CPI_Data')

In [7]:
print(dataset_melt.head())

   Year Month  CPI_Data
0  1913   Jan       9.8
1  1914   Jan      10.0
2  1915   Jan      10.1
3  1916   Jan      10.4
4  1917   Jan      11.7


In [8]:
dataset_melt = dataset_melt.dropna(axis=0, how='any')
X_train = dataset_melt.iloc[:,2:3].values
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler.fit_transform(X_train)

In [9]:
print(X_train_scaled)

[[3.27765687e-04]
 [9.83297061e-04]
 [1.31106275e-03]
 ...
 [8.82024019e-01]
 [9.41005454e-01]
 [9.73614862e-01]]


### Building the RNN
##### Creating a 24 Timestamps (or two years)

In [10]:
X_train = []
y_train = []
timestamp = 24
for i in range(timestamp, len(X_train_scaled)):
    X_train.append(X_train_scaled[i-timestamp:i,0])
    y_train.append(X_train_scaled[i,0])
X_train = np.array(X_train)
y_train = np.array(y_train)

In [11]:
print(X_train)

[[3.27765687e-04 9.83297061e-04 1.31106275e-03 ... 1.14717990e-02
  1.27828618e-02 1.34383932e-02]
 [9.83297061e-04 1.31106275e-03 2.29435981e-03 ... 1.27828618e-02
  1.34383932e-02 1.44216902e-02]
 [1.31106275e-03 2.29435981e-03 6.55531374e-03 ... 1.34383932e-02
  1.44216902e-02 1.47494559e-02]
 ...
 [4.96892781e-01 5.05414689e-01 5.19836379e-01 ... 7.91662296e-01
  8.10479325e-01 8.21951124e-01]
 [5.05414689e-01 5.19836379e-01 5.38519024e-01 ... 8.10479325e-01
  8.21951124e-01 8.82024019e-01]
 [5.19836379e-01 5.38519024e-01 5.47368697e-01 ... 8.21951124e-01
  8.82024019e-01 9.41005454e-01]]


In [12]:
print(y_train)

[0.01442169 0.01474946 0.01409392 ... 0.88202402 0.94100545 0.97361486]


#### Reshaping the Data

In [13]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

#### Building the RNN
Step 1 Building the LSTM

In [14]:
regressor = Sequential()
# Adding the LSTM
regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
regressor.add(Dropout(0.2))
# Adding Second LSTM
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
# Adding Third LSTM 
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
# Adding 4th LSTM
regressor.add(LSTM(units=50, return_sequences=False))
regressor.add(Dropout(0.2))

#### Adding the output Layer

In [15]:
regressor.add(Dense(units=1))

#### Compile and Fitting

In [16]:
regressor.compile(optimizer='adam', loss='mean_squared_error')

In [17]:
regressor.fit(X_train, y_train,epochs=100,batch_size=10)

Epoch 1/100
132/132 [==============================] - 8s 21ms/step - loss: 0.0510
Epoch 2/100
132/132 [==============================] - 3s 19ms/step - loss: 0.0302
Epoch 3/100
132/132 [==============================] - 3s 19ms/step - loss: 0.0203
Epoch 4/100
132/132 [==============================] - 3s 19ms/step - loss: 0.0187
Epoch 5/100
132/132 [==============================] - 3s 19ms/step - loss: 0.0174
Epoch 6/100
132/132 [==============================] - 2s 19ms/step - loss: 0.0156
Epoch 7/100
132/132 [==============================] - 3s 20ms/step - loss: 0.0182
Epoch 8/100
132/132 [==============================] - 3s 20ms/step - loss: 0.0174
Epoch 9/100
132/132 [==============================] - 3s 22ms/step - loss: 0.0156
Epoch 10/100
132/132 [==============================] - 3s 20ms/step - loss: 0.0166
Epoch 11/100
132/132 [==============================] - 3s 20ms/step - loss: 0.0127
Epoch 12/100
132/132 [==============================] - 3s 19ms/step - loss: 0.0130
E

In [18]:
regressor.save("models/CPI_Predictor")

INFO:tensorflow:Assets written to: models/CPI_Predictor\assets


INFO:tensorflow:Assets written to: models/CPI_Predictor\assets


### Prediction of September 2024 CPI

In [19]:
cpi_predictor = load_model("models/CPI_Predictor")

In [20]:
X_test = dataset_melt.iloc[-timestamp:, 2:3].values

In [21]:
print(X_test)

[[174.   ]
 [176.7  ]
 [180.9  ]
 [184.3  ]
 [190.3  ]
 [196.8  ]
 [201.8  ]
 [210.036]
 [210.228]
 [215.949]
 [219.179]
 [225.672]
 [229.601]
 [233.049]
 [234.812]
 [236.525]
 [241.432]
 [246.524]
 [251.233]
 [256.974]
 [260.474]
 [278.802]
 [296.797]
 [306.746]]


In [22]:
X_test_scaled = scaler.transform(X_test)

In [23]:
X_test = []
X_test.append(X_test_scaled)
X_test = np.array(X_test)

In [24]:
print(X_test)

[[[0.53851902]
  [0.5473687 ]
  [0.56113486]
  [0.57227889]
  [0.59194483]
  [0.6132496 ]
  [0.62963788]
  [0.65663267]
  [0.65726198]
  [0.67601345]
  [0.68660028]
  [0.70788211]
  [0.72076002]
  [0.73206138]
  [0.73783989]
  [0.74345452]
  [0.75953798]
  [0.77622781]
  [0.7916623 ]
  [0.81047932]
  [0.82195112]
  [0.88202402]
  [0.94100545]
  [0.97361486]]]


In [25]:
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [26]:
Sept_CPI = cpi_predictor.predict(X_test)

1/1 [==============================] - 1s 1s/step


In [27]:
Sept_CPI = scaler.inverse_transform(Sept_CPI)

In [28]:
print(Sept_CPI)

[[126.924515]]
